In [73]:
import glob
import numpy as np
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

CASH = 100000
start_equity = 0
final_equity = 0

In [74]:
path = "./*.csv"
files = glob.glob(path)

start_equity = CASH * len(files)

names = [x[2:-4] for x in files]

stocks = {
    name: stock for name, stock in zip(names, [pd.read_csv(file, parse_dates=['Date']) for file in files])
}

for stock in stocks.values():
    stock.set_index('Date',inplace=True)
    stock.sort_index(inplace=True)

things to do:
- reverse stock dates?
- copy over SMA
- figure out how to measure market change (2.54x increase over 5y?)

In [57]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [58]:
class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [59]:
stats = {}
for name, stock in stocks.items():
    bt = Backtest(stock, SmaCross, cash=100000, commission=.0025)
    stats[name] = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Return [%]',
                    constraint=lambda param: param.n1 < param.n2)

In [76]:
for name, stock in stats.items():
    final_equity += stock["Equity Final [$]"]

In [78]:
print(start_equity)
print(final_equity)
print('{:.2%}'.format(final_equity/start_equity))

500000
2504101.020489503
500.82%
